### Data loading
This book build on top of previous notebook basic_notebook: that served to load the data and get a understanding for it. In particular this notebook will focus on feature engineering. Note: this book will only consider data from the following csv files: train, test and structures


In [1]:
import pandas as pd

input_folder = './input'

train = pd.read_csv(f'{input_folder}/train.csv')
test = pd.read_csv(f'{input_folder}/test.csv')
structures = pd.read_csv(f'{input_folder}/structures.csv')

In [2]:
f'Train shape: {train.shape}'

'Train shape: (4658147, 6)'

In [3]:
train.head(5)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [4]:
f'Test shape: {test.shape}'

'Test shape: (2505542, 5)'

In [5]:
f'Structures shape: {structures.shape}'

'Structures shape: (2358657, 6)'

In [6]:
structures.head(5)

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


From the first basic notebook we have (basic_notebook.ipynb)

- There are 85003 distinct molecules in train data.
- There are 45772 distinct molecules in test data.
- Largest molecule contains upto 29 atoms.
- There are 8 unique coupling types: ['1JHC' '2JHH' '1JHN' '2JHN' '2JHC' '3JHH' '3JHC' '3JHN']
- There are 5 unique atom types: ['C' 'H' 'N' 'O' 'F']
      
- Molecule overlap bewteen training and test data sets: 0
- Train molecules described in structures file: 100.0%
- Test molecules described in structures file: 100.0%

#### Any missing values?

From the first basic notebook we have (basic_notebook.ipynb) we know the answer is no.

#### Merging data sources into one training data set and one test data set 

Code for joining data is from this kernel: https://www.kaggle.com/seriousran/just-speed-up-calculate-distance-from-benchmark

In [7]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

In [8]:
train = map_atom_info(train, 0)
train = map_atom_info(train, 1)

In [9]:
train.head(10)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.2541,H,1.011731,1.463751,0.000277,H,-0.540815,1.447527,-0.876644
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.2548,H,1.011731,1.463751,0.000277,H,-0.523814,1.437933,0.906397
7,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,H,-0.540815,1.447527,-0.876644,C,-0.012698,1.085804,0.008001
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.2543,H,-0.540815,1.447527,-0.876644,H,-0.523814,1.437933,0.906397
9,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,H,-0.523814,1.437933,0.906397,C,-0.012698,1.085804,0.008001


In [10]:
test = map_atom_info(test, 0)
test = map_atom_info(test, 1)

In [11]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,4658147,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0
1,4658148,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0
2,4658149,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0
3,4658150,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0
4,4658151,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0


### Feature engineering
#### Distances between atoms

In [12]:
train_vec_0 = train[['x_0', 'y_0', 'z_0']].values
train_vec_1 = train[['x_1', 'y_1', 'z_1']].values
test_vec_0 = test[['x_0', 'y_0', 'z_0']].values
test_vec_1 = test[['x_1', 'y_1', 'z_1']].values

In [13]:
import numpy as np

train['dist'] = np.linalg.norm(train_vec_0 - train_vec_1, axis=1)
test['dist'] = np.linalg.norm(test_vec_0 - test_vec_1, axis=1)

In [14]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,1.091953
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,1.783120
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,1.783147
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,1.783157
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,1.091952


#### No of atomic bond distances

In [15]:
train['atomic_bond_distances'] = train['type'].apply(lambda x: int(x[0]))
test['atomic_bond_distances'] = train['type'].apply(lambda x: int(x[0]))

In [16]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist,atomic_bond_distances
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,1.091953,1
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,1.783120,2
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,1.783147,2
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,1.783157,2
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,1.091952,1


#### Make a copy of data for first training

In [21]:
columns_to_use_v1 = [
    'molecule_name',
    'atom_index_0',
    'atom_index_1',    
    'type',
    'atom_0',
    'x_0',
    'y_0',
    'z_0',
    'atom_1',
    'x_1',
    'y_1',
    'z_1',
    'dist',
    'atomic_bond_distances'
]

For the above columns I will consider atom_index's ordinal data together with no_of_atomic_bond_distances. I will argue that atom's are pure labels, that is, nominal data.

LabelEncoder could be used for the ordinal data but as they are already ordered integers we will leave them be. One Hot Encoding is used for nominal data.

In [22]:
train_X = train[columns_to_use_v1].copy()
train_y = train['scalar_coupling_constant']
test_X = test[columns_to_use_v1].copy()

In [23]:
train_X.head()

,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist,atomic_bond_distances
0,dsgdb9nsd_000001,1,0,1JHC,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,1.091953,1
1,dsgdb9nsd_000001,1,2,2JHH,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,1.783120,2
2,dsgdb9nsd_000001,1,3,2JHH,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,1.783147,2
3,dsgdb9nsd_000001,1,4,2JHH,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,1.783157,2
4,dsgdb9nsd_000001,2,0,1JHC,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,1.091952,1


In [24]:
test_X.head()

,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist,atomic_bond_distances
0,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,2.261178,1
1,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,1.062099,2
2,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,3.323277,2
3,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,1.062099,2
4,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,2.261178,1


#### Data splitting

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, random_state = 13, test_size = 0.25)


### The model

In [26]:
from catboost import CatBoostRegressor

In [27]:
categorical_features_indices = [0,3,4,8] #np.where(train_X.dtypes != np.float)[0]
print(categorical_features_indices)

[0, 3, 4, 8]


In [28]:
basic_model = CatBoostRegressor(task_type = "GPU", devices='0', gpu_ram_part=0.95, gpu_cat_features_storage='GpuRam',  iterations=12000, random_seed=42, loss_function='MAE', custom_metric=['MAE'], eval_metric='MAE', grow_policy='SymmetricTree', border_count=255, random_strength=2, learning_rate=0.03, l2_leaf_reg=6, depth=3)
# Old model: bestTest = 3.697536446, bestIteration = 11999 using: iterations=12000, random_seed=42, loss_function='MAE', custom_metric=['MAE'], eval_metric='MAE', grow_policy='SymmetricTree', border_count=255, random_strength=2, learning_rate=0.03, l2_leaf_reg=6, depth=3
# v1 model: bestTest ~ 2.7 bestIteration = 11999 using: iterations=12000, random_seed=42, loss_function='MAE', custom_metric=['MAE'], eval_metric='MAE', grow_policy='SymmetricTree', border_count=255, random_strength=2, learning_rate=0.03, l2_leaf_reg=6, depth=3
# v1 model: bestTest = 2.53 bestIteration = 19999 using: iterations=20000, random_seed=42, loss_function='MAE', custom_metric=['MAE'], eval_metric='MAE', grow_policy='SymmetricTree', border_count=255, random_strength=2, learning_rate=0.03, l2_leaf_reg=6, depth=3
# this model: bestTest ~ 2.62 bestIteration = 11999 using: iterations=12000, random_seed=42, loss_function='MAE', custom_metric=['MAE'], eval_metric='MAE', grow_policy='SymmetricTree', border_count=255, random_strength=2, learning_rate=0.03, l2_leaf_reg=6, depth=3


In [29]:
basic_model.fit(X_train, y_train, 
          cat_features=categorical_features_indices, 
          eval_set=(X_valid, y_valid),
          logging_level='Verbose', plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 18.3685964	test: 18.4074117	best: 18.4074117 (0)	total: 123ms	remaining: 24m 33s
1:	learn: 18.3589113	test: 18.3977237	best: 18.3977237 (1)	total: 232ms	remaining: 23m 10s
2:	learn: 18.3492857	test: 18.3880753	best: 18.3880753 (2)	total: 340ms	remaining: 22m 41s
3:	learn: 18.3397013	test: 18.3784835	best: 18.3784835 (3)	total: 456ms	remaining: 22m 48s
4:	learn: 18.3301971	test: 18.3689398	best: 18.3689398 (4)	total: 572ms	remaining: 22m 53s
5:	learn: 18.3207009	test: 18.3594338	best: 18.3594338 (5)	total: 679ms	remaining: 22m 37s
6:	learn: 18.3112666	test: 18.3499863	best: 18.3499863 (6)	total: 795ms	remaining: 22m 42s
7:	learn: 18.3018688	test: 18.3406264	best: 18.3406264 (7)	total: 912ms	remaining: 22m 46s
8:	learn: 18.2925627	test: 18.3312836	best: 18.3312836 (8)	total: 1.03s	remaining: 22m 48s
9:	learn: 18.2832726	test: 18.3219855	best: 18.3219855 (9)	total: 1.13s	remaining: 22m 40s
10:	learn: 18.2740581	test: 18.3127303	best: 18.3127303 (10)	total: 1.24s	remaining: 22m 3

90:	learn: 17.6547228	test: 17.6928582	best: 17.6928582 (90)	total: 9.96s	remaining: 21m 43s
91:	learn: 17.6481805	test: 17.6863217	best: 17.6863217 (91)	total: 10.1s	remaining: 21m 42s
92:	learn: 17.6416727	test: 17.6798213	best: 17.6798213 (92)	total: 10.2s	remaining: 21m 41s
93:	learn: 17.6352232	test: 17.6733380	best: 17.6733380 (93)	total: 10.3s	remaining: 21m 40s
94:	learn: 17.6287485	test: 17.6668599	best: 17.6668599 (94)	total: 10.4s	remaining: 21m 39s
95:	learn: 17.6223356	test: 17.6604367	best: 17.6604367 (95)	total: 10.5s	remaining: 21m 39s
96:	learn: 17.6159400	test: 17.6540290	best: 17.6540290 (96)	total: 10.6s	remaining: 21m 38s
97:	learn: 17.6095592	test: 17.6476334	best: 17.6476334 (97)	total: 10.7s	remaining: 21m 38s
98:	learn: 17.6032024	test: 17.6412961	best: 17.6412961 (98)	total: 10.8s	remaining: 21m 38s
99:	learn: 17.5968789	test: 17.6349399	best: 17.6349399 (99)	total: 10.9s	remaining: 21m 37s
100:	learn: 17.5905805	test: 17.6286352	best: 17.6286352 (100)	total: 

179:	learn: 17.1526919	test: 17.1903615	best: 17.1903615 (179)	total: 19.3s	remaining: 21m 10s
180:	learn: 17.1477263	test: 17.1853999	best: 17.1853999 (180)	total: 19.5s	remaining: 21m 10s
181:	learn: 17.1427767	test: 17.1804434	best: 17.1804434 (181)	total: 19.6s	remaining: 21m 9s
182:	learn: 17.1377904	test: 17.1754818	best: 17.1754818 (182)	total: 19.7s	remaining: 21m 9s
183:	learn: 17.1328958	test: 17.1705476	best: 17.1705476 (183)	total: 19.8s	remaining: 21m 9s
184:	learn: 17.1279530	test: 17.1656100	best: 17.1656100 (184)	total: 19.9s	remaining: 21m 9s
185:	learn: 17.1230332	test: 17.1607051	best: 17.1607051 (185)	total: 20s	remaining: 21m 9s
186:	learn: 17.1181626	test: 17.1557915	best: 17.1557915 (186)	total: 20.1s	remaining: 21m 8s
187:	learn: 17.1132347	test: 17.1508849	best: 17.1508849 (187)	total: 20.2s	remaining: 21m 8s
188:	learn: 17.1083664	test: 17.1460074	best: 17.1460074 (188)	total: 20.3s	remaining: 21m 8s
189:	learn: 17.1034786	test: 17.1411402	best: 17.1411402 (18

267:	learn: 16.7468607	test: 16.7840129	best: 16.7840129 (267)	total: 28.6s	remaining: 20m 52s
268:	learn: 16.7424710	test: 16.7796060	best: 16.7796060 (268)	total: 28.7s	remaining: 20m 52s
269:	learn: 16.7380870	test: 16.7751767	best: 16.7751767 (269)	total: 28.8s	remaining: 20m 52s
270:	learn: 16.7336629	test: 16.7707870	best: 16.7707870 (270)	total: 28.9s	remaining: 20m 52s
271:	learn: 16.7292594	test: 16.7664059	best: 16.7664059 (271)	total: 29s	remaining: 20m 52s
272:	learn: 16.7248937	test: 16.7620007	best: 16.7620007 (272)	total: 29.1s	remaining: 20m 52s
273:	learn: 16.7205532	test: 16.7576144	best: 16.7576144 (273)	total: 29.3s	remaining: 20m 51s
274:	learn: 16.7161772	test: 16.7532401	best: 16.7532401 (274)	total: 29.4s	remaining: 20m 51s
275:	learn: 16.7118368	test: 16.7488762	best: 16.7488762 (275)	total: 29.5s	remaining: 20m 51s
276:	learn: 16.7074653	test: 16.7444933	best: 16.7444933 (276)	total: 29.6s	remaining: 20m 51s
277:	learn: 16.7031237	test: 16.7401568	best: 16.740

...

11590:	learn: 2.6322537	test: 2.6305317	best: 2.6305317 (11590)	total: 20m 29s	remaining: 43.4s
11591:	learn: 2.6322257	test: 2.6305044	best: 2.6305044 (11591)	total: 20m 29s	remaining: 43.3s
11592:	learn: 2.6321770	test: 2.6304476	best: 2.6304476 (11592)	total: 20m 30s	remaining: 43.2s
11593:	learn: 2.6321378	test: 2.6304025	best: 2.6304025 (11593)	total: 20m 30s	remaining: 43.1s
11594:	learn: 2.6321129	test: 2.6303776	best: 2.6303776 (11594)	total: 20m 30s	remaining: 43s
11595:	learn: 2.6320883	test: 2.6303518	best: 2.6303518 (11595)	total: 20m 30s	remaining: 42.9s
11596:	learn: 2.6320597	test: 2.6303250	best: 2.6303250 (11596)	total: 20m 30s	remaining: 42.8s
11597:	learn: 2.6320356	test: 2.6303001	best: 2.6303001 (11597)	total: 20m 30s	remaining: 42.7s
11598:	learn: 2.6320099	test: 2.6302756	best: 2.6302756 (11598)	total: 20m 30s	remaining: 42.5s
11599:	learn: 2.6319621	test: 2.6302290	best: 2.6302290 (11599)	total: 20m 30s	remaining: 42.4s
11600:	learn: 2.6319360	test: 2.6302063	be

11676:	learn: 2.6295643	test: 2.6278021	best: 2.6278021 (11676)	total: 20m 39s	remaining: 34.3s
11677:	learn: 2.6295405	test: 2.6277780	best: 2.6277780 (11677)	total: 20m 39s	remaining: 34.2s
11678:	learn: 2.6295073	test: 2.6277383	best: 2.6277383 (11678)	total: 20m 39s	remaining: 34.1s
11679:	learn: 2.6294830	test: 2.6277123	best: 2.6277123 (11679)	total: 20m 39s	remaining: 34s
11680:	learn: 2.6294426	test: 2.6276724	best: 2.6276724 (11680)	total: 20m 39s	remaining: 33.9s
11681:	learn: 2.6294160	test: 2.6276460	best: 2.6276460 (11681)	total: 20m 39s	remaining: 33.8s
11682:	learn: 2.6293945	test: 2.6276205	best: 2.6276205 (11682)	total: 20m 39s	remaining: 33.6s
11683:	learn: 2.6293573	test: 2.6275878	best: 2.6275878 (11683)	total: 20m 40s	remaining: 33.5s
11684:	learn: 2.6293147	test: 2.6275455	best: 2.6275455 (11684)	total: 20m 40s	remaining: 33.4s
11685:	learn: 2.6292872	test: 2.6275213	best: 2.6275213 (11685)	total: 20m 40s	remaining: 33.3s
11686:	learn: 2.6292222	test: 2.6274584	be

11763:	learn: 2.6266787	test: 2.6249269	best: 2.6249269 (11763)	total: 20m 48s	remaining: 25s
11764:	learn: 2.6266535	test: 2.6249052	best: 2.6249052 (11764)	total: 20m 48s	remaining: 24.9s
11765:	learn: 2.6266292	test: 2.6248820	best: 2.6248820 (11765)	total: 20m 48s	remaining: 24.8s
11766:	learn: 2.6265911	test: 2.6248415	best: 2.6248415 (11766)	total: 20m 48s	remaining: 24.7s
11767:	learn: 2.6265656	test: 2.6248183	best: 2.6248183 (11767)	total: 20m 48s	remaining: 24.6s
11768:	learn: 2.6265425	test: 2.6247936	best: 2.6247936 (11768)	total: 20m 48s	remaining: 24.5s
11769:	learn: 2.6265164	test: 2.6247678	best: 2.6247678 (11769)	total: 20m 49s	remaining: 24.4s
11770:	learn: 2.6264918	test: 2.6247434	best: 2.6247434 (11770)	total: 20m 49s	remaining: 24.3s
11771:	learn: 2.6264305	test: 2.6246789	best: 2.6246789 (11771)	total: 20m 49s	remaining: 24.2s
11772:	learn: 2.6264085	test: 2.6246547	best: 2.6246547 (11772)	total: 20m 49s	remaining: 24.1s
11773:	learn: 2.6263430	test: 2.6245901	be

11849:	learn: 2.6240053	test: 2.6222643	best: 2.6222643 (11849)	total: 20m 57s	remaining: 15.9s
11850:	learn: 2.6239806	test: 2.6222381	best: 2.6222381 (11850)	total: 20m 57s	remaining: 15.8s
11851:	learn: 2.6239583	test: 2.6222155	best: 2.6222155 (11851)	total: 20m 57s	remaining: 15.7s
11852:	learn: 2.6239323	test: 2.6221913	best: 2.6221913 (11852)	total: 20m 57s	remaining: 15.6s
11853:	learn: 2.6239082	test: 2.6221713	best: 2.6221713 (11853)	total: 20m 57s	remaining: 15.5s
11854:	learn: 2.6238790	test: 2.6221382	best: 2.6221382 (11854)	total: 20m 57s	remaining: 15.4s
11855:	learn: 2.6238550	test: 2.6221144	best: 2.6221144 (11855)	total: 20m 58s	remaining: 15.3s
11856:	learn: 2.6238155	test: 2.6220655	best: 2.6220655 (11856)	total: 20m 58s	remaining: 15.2s
11857:	learn: 2.6237917	test: 2.6220429	best: 2.6220429 (11857)	total: 20m 58s	remaining: 15.1s
11858:	learn: 2.6237514	test: 2.6220011	best: 2.6220011 (11858)	total: 20m 58s	remaining: 15s
11859:	learn: 2.6237273	test: 2.6219755	be

11938:	learn: 2.6216143	test: 2.6198777	best: 2.6198777 (11938)	total: 21m 6s	remaining: 6.47s
11939:	learn: 2.6215937	test: 2.6198539	best: 2.6198539 (11939)	total: 21m 6s	remaining: 6.37s
11940:	learn: 2.6215554	test: 2.6198150	best: 2.6198150 (11940)	total: 21m 6s	remaining: 6.26s
11941:	learn: 2.6215322	test: 2.6197925	best: 2.6197925 (11941)	total: 21m 7s	remaining: 6.15s
11942:	learn: 2.6215110	test: 2.6197714	best: 2.6197714 (11942)	total: 21m 7s	remaining: 6.05s
11943:	learn: 2.6214701	test: 2.6197300	best: 2.6197300 (11943)	total: 21m 7s	remaining: 5.94s
11944:	learn: 2.6214480	test: 2.6197100	best: 2.6197100 (11944)	total: 21m 7s	remaining: 5.83s
11945:	learn: 2.6214217	test: 2.6196873	best: 2.6196873 (11945)	total: 21m 7s	remaining: 5.73s
11946:	learn: 2.6213991	test: 2.6196654	best: 2.6196654 (11946)	total: 21m 7s	remaining: 5.62s
11947:	learn: 2.6213576	test: 2.6196297	best: 2.6196297 (11947)	total: 21m 7s	remaining: 5.52s
11948:	learn: 2.6213352	test: 2.6196057	best: 2.61

In [30]:
X_valid.head()

,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist,atomic_bond_distances
3196247,dsgdb9nsd_092878,14,5,3JHC,H,1.994132,-0.008963,-2.755838,C,3.159101,0.387629,0.493604,3.474667,3
3223883,dsgdb9nsd_093594,11,1,2JHC,H,-0.559436,1.919815,0.865642,C,0.021745,0.014319,-0.006325,2.174630,2
1128197,dsgdb9nsd_038213,15,4,1JHC,H,-2.196463,-0.119074,0.549347,C,-1.438739,-0.611970,-0.069574,1.095516,1
327588,dsgdb9nsd_011396,14,18,3JHH,H,1.001034,-1.296349,4.118188,H,0.613491,-3.570417,2.720083,2.697456,3
271005,dsgdb9nsd_009654,11,4,2JHC,H,1.511599,-0.019544,-1.778025,C,0.188195,-1.804189,-1.586486,2.230032,2


In [31]:
y_pred = basic_model.predict(X_valid)
print(y_pred[:10])
# v1: iter=1000 [ 4.07469207 -1.62194332 14.99980471  3.0258367  -0.41768674 -1.35653131 2.10979457 14.99980471  3.12673688  3.81138391]
# v1: iter=3000 [ 4.83265408 -1.96371974 44.97979771  3.05219204 -0.39719858 -1.55467009 1.81603856 44.97979771  3.95422958  4.05474575]
# v1: iter=20000 [ 5.4143218  -2.08383771 85.22568139  2.08890557 -0.26324311 -2.25495008 1.57246787 95.32529635  5.40039232  3.9473724 ]
# this:
# 21: iter=12000 [ 4.33806061 -2.18875398 84.05569871  2.47354582 -0.12624203 -1.96473346 1.66958408 96.77799089  4.65348689  4.06718966]

[ 4.33806061 -2.18875398 84.05569871  2.47354582 -0.12624203 -1.96473346
  1.66958408 96.77799089  4.65348689  4.06718966]


In [33]:
# Competition metric from here https://www.kaggle.com/uberkinder/efficient-metric

def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()


In [34]:
score = group_mean_log_mae(y_valid, y_pred, X_valid['type'])
print(f'score {score}')
# v1: 1000  iter 1.52793
# v1: 3000  iter 1.25545
# v1: 20000 iter 0.82817
# this: 12000 iter 0.83715



score 0.8371558920681527


Above score currently takes you to top 85%

### Let's submit it!
Utilty code for generating csv file of test predictions

In [35]:
test_X.head()

,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist,atomic_bond_distances
0,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,2.261178,1
1,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,1.062099,2
2,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,3.323277,2
3,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,1.062099,2
4,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,2.261178,1


In [36]:
predictions = basic_model.predict(test_X)

submission = pd.read_csv('./input/sample_submission.csv')
submission['scalar_coupling_constant'] = predictions
submission.to_csv('./results/basic_feature_engineering_CAT_v2.csv', index=False)
print(submission.shape)

(2505542, 2)


I DO NOT UNDERSTAND WHY THIS RESULTS IN A SCORE OF 2.423 ON KAGGLE WOULD HAVE EXPECTED AROUND ~ 0.85!!!